In [1]:
import os
import pandas as pd
import sys
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf

sys.path.append("../../code/src")

# MB-1004-GE-2Hist

In [2]:
raw_data = pd.read_csv('raw_data/MBdata_33CLINwMiss_1KfGE_1KfCNA.csv')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
all_genes_df = pd.read_csv('raw_data/all_genes.csv')

In [ ]:
df_4 = all_genes_df[['METABRIC_ID','CDH1', 'MKI67', 'FOXA1', 'PTEN']]

In [ ]:
df_4.rename(columns={"CDH1": "GE_CDH1", 'MKI67':'GE_MKI67', 'FOXA1':'GE_FOXA1', 'PTEN':'GE_PTEN'}, inplace=True)

In [ ]:
df_4_raw_data = pd.merge(df_4, raw_data, on='METABRIC_ID')

In [ ]:
df_4_raw_data_2Hist = df_4_raw_data[(df_4_raw_data['Histological_Type'] == 'IDC') | (df_4_raw_data['Histological_Type'] == 'ILC')].copy()

In [ ]:
df_4_raw_data_2Hist['Histological_Type'].replace({"IDC":0,"ILC":1}, inplace=True)

In [ ]:
genes_1000_2Hist = df_4_raw_data_2Hist.iloc[:,38:1038]
genes_4_2Hist = df_4_raw_data_2Hist.iloc[:,1:5]
labels_2Hist = df_4_raw_data_2Hist.loc[:,'Histological_Type']

In [ ]:
MB_1004_GE_2Hist = pd.concat([genes_1000_2Hist, genes_4_2Hist, labels_2Hist ], axis=1)

In [ ]:
MB_1004_GE_2Hist["Histological_Type"].value_counts()

In [ ]:
MB_1004_GE_2Hist.to_csv('MB-1004-GE-2Hist.csv', index=False)

## save

In [ ]:
dataset_name = 'MB-1004-GE-2Hist'
target_col_name = 'Histological_Type'
data = MB_1004_GE_2Hist

In [ ]:
from model.generation.helpers import init_dataset_dir
path_to_data_folder = '../'
init_dataset_dir.run(dataset_name=dataset_name, path_to_data_folder=path_to_data_folder)

In [ ]:
data_path = '../' + dataset_name + '/'

In [ ]:
data.to_csv(data_path + 'data.csv', index=False)

# MB-ClinP-ER

In [ ]:
raw_data = pd.read_csv('raw_data/MBdata_33CLINwMiss_1KfGE_1KfCNA.csv')

In [ ]:
clin = raw_data.iloc[:,2:19]

In [ ]:
clin.drop(["Date_Of_Diagnosis", "Last_Followup_Status", "Breast_Surgery", "ER_Status"], axis =1, inplace=True)

In [ ]:
clin['Age_At_Diagnosis'] = clin['Age_At_Diagnosis'].astype(float)
clin["Breast_Tumour_Laterality"] = to_categorical(clin["Breast_Tumour_Laterality"])
clin['NPI'] = clin['NPI'].astype(float)
clin["Inferred_Menopausal_State"] = to_categorical(clin["Inferred_Menopausal_State"])
clin['Lymph_Nodes_Positive'] = clin['Lymph_Nodes_Positive'].astype(int)

In [ ]:
clin["CT"] = to_categorical(clin["CT"])
clin["HT"] = to_categorical(clin["HT"])
clin["RT"] = to_categorical(clin["RT"])

In [ ]:
clin["Grade"].replace("?", np.NaN, inplace=True)
clin.fillna(clin['Grade'].value_counts().index[0], inplace=True)
clin['Grade'] = clin['Grade'].astype(int)

In [ ]:
clin["Size"].replace("?", np.NaN, inplace=True)
clin['Size'] = clin['Size'].astype(float)
clin['Size'].fillna(clin['Size'].mean(), inplace=True)

In [ ]:
clin["Histological_Type"] = to_categorical(clin["Histological_Type"])

In [ ]:
clin["Stage"].replace("?", np.NaN, inplace=True)
clin.fillna(clin['Stage'].value_counts().index[0], inplace=True)
clin['Stage'] = clin['Stage'].astype(int)

In [ ]:
clin["Cellularity"] = to_categorical(clin["Cellularity"])

In [ ]:
er_data_df = raw_data.loc[:,"ER_Expr"]
er_data_df.replace({"-":0,"+":1}, inplace=True)
clin["ER_Expr"] = er_data_df.values

## save

In [ ]:
dataset_name = 'MB-ClinP-ER'
target_col_name = 'ER_Expr'

In [ ]:
init_dataset_dir.run(dataset_name=dataset_name, path_to_data_folder=path_to_data_folder)

In [ ]:
clin.to_csv(data_path + 'data.csv', index=False)

# MB-GE-ER

In [ ]:
raw_data = pd.read_csv('raw_data/MBdata_33CLINwMiss_1KfGE_1KfCNA.csv')
ge = raw_data.iloc[:,34:1034]

In [ ]:
er = raw_data.loc[:, "ER_Expr"]

In [ ]:
MB_GE_ER = pd.concat([ge, er], axis=1)

In [ ]:
MB_GE_ER["ER_Expr"].replace({'+':1, '-': 0}, inplace=True)

In [ ]:
MB_GE_ER.to_csv('MB-GE-ER.csv', index=False)

## save

In [ ]:
dataset_name = 'MB-GE-ER'
target_col_name = 'ER_Expr'
data = MB_GE_ER

In [ ]:
from model.generation.helpers import init_dataset_dir
path_to_data_folder = '../'
init_dataset_dir.run(dataset_name=dataset_name, path_to_data_folder=path_to_data_folder)

In [ ]:
data_path = '../' + dataset_name + '/'

In [ ]:
data.to_csv(data_path + 'data.csv', index=False)

# MB-GE-ClinP-ER

In [ ]:
GE_data = pd.read_csv("../MB-GE-ER/data.csv")
ClinP_data = pd.read_csv("../MB-ClinP-ER/data.csv")

In [ ]:
GE_data = GE_data.drop(columns=["ER_Expr"])

In [ ]:
GE_ClinP_data = pd.concat([GE_data, ClinP_data], axis=1)

##  save

In [ ]:
dataset_name = 'MB-GE-ClinP-ER'
target_col_name = 'ER_Expr'

In [ ]:
from model.generation.helpers import init_dataset_dir
path_to_data_folder = '../'
init_dataset_dir.run(dataset_name=dataset_name, path_to_data_folder=path_to_data_folder)

In [ ]:
data_path = '../' + dataset_name + '/'

In [ ]:
GE_ClinP_data.to_csv(data_path + 'data.csv', index=False)